In [1]:
'''
This code opens the Microsoft Academic Graph Dataset and trains HGT
Based on code provided by original HGT paper
'''
import torch
from hgt import *
from hgt_utils import *
from model import *
from ogb.nodeproppred import PygNodePropPredDataset
from ogb.nodeproppred import Evaluator
import multiprocessing as mp
import argparse
import numpy as np
import time
import pandas as pd
import matplotlib as plt
import seaborn as sb

print("Microsoft Academic Graph Dataset Experiment")

Microsoft Academic Graph Dataset Experiment


In [2]:
'''
Data Preprocessing
ogbn-mag only comes with paper node features, thus for other nodes types we take the average
of connected paper nodes as input features. 
'''
print("Begin Data Preprocessing")
print("")
print("Retrieving Data from Open Graph Benchmark ...")

# Get dataset using Pytorch Geometric Loader
dataset = PygNodePropPredDataset(name='ogbn-mag')
print("... Retrieval complete")
data = dataset[0] # pyg graph object

Begin Data Preprocessing

Retrieving Data from Open Graph Benchmark ...
... Retrieval complete


In [3]:
evaluator = Evaluator(name='ogbn-mag')
# Preparing Graph
graph, y, train_paper, valid_paper, test_paper = prepare_graph(data, dataset)

Populating edge lists into Graph object
('author', 'affiliated_with', 'institution')
('author', 'writes', 'paper')
('paper', 'cites', 'paper')
('paper', 'has_topic', 'field_of_study')

Reformatting edge lists and computing node degrees
institution author affiliated_with 8740
author institution rev_affiliated_with 852987
author paper rev_writes 1134649
paper author writes 736389
paper paper cites 629169
paper paper rev_cites 617924
paper field_of_study rev_has_topic 736389
field_of_study paper has_topic 59965

Constructing node feature vectors for each node type in graph
author
field_of_study
institution
paper

Constructing Node features for institutions

Splitting dataset into train, val and test

Creating Masks

Preprocessing complete


In [4]:
'''
Creating Model
'''
print("Creating Model")
hgt_GNN = HGTModel(len(graph.node_feature['paper'][0]), # input_dim
                   256,                                 # hidden_dim
                   len(graph.get_types()),              # num_node_types
                   len(graph.get_meta_graph()),         # num_edge_types
                   8,                                   # num_heads
                   4,                                   # num_layers
                   0.2,                                 # dropout
                   prev_norm = True,                    # normalization on all but last layer
                   last_norm = False,                   # normalization on last layer
                   use_rte = False)                     # use relative temporal encoding 
classifier = Classifier(256, graph.y.max()+1)

print(f'Classifier output dim is: {graph.y.max()+1}')

HGT_classifier = nn.Sequential(hgt_GNN, classifier)

print(HGT_classifier)

Creating Model
Classifier output dim is: 349
Sequential(
  (0): HGTModel(
    (adapt_features): ModuleList(
      (0-3): 4 x Linear(in_features=129, out_features=256, bias=True)
    )
    (hgt_layers): ModuleList(
      (0-3): 4 x HGTLayer()
    )
    (drop): Dropout(p=0.2, inplace=False)
  )
  (1): Classifier(n_hid=256, n_out=349)
)


In [5]:
'''
Preprocessing data
'''
batch_number = 32 # number of sampled graphs for each epoch
batch_size = 128
num_epochs = 10
num_workers = 8
clip = 1.0
sample_depth = 6
sample_width = 520
plot = False # True or false to plot data
target_nodes = np.arange(len(graph.node_feature['paper']))

# Negative Log Likelihood Loss
criterion = nn.NLLLoss()

# Get list of model parameters w/ associated names
parameters_optimizer = list(HGT_classifier.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in parameters_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in parameters_optimizer if any(nd in n for nd in no_decay)],     'weight_decay': 0.0}
]
# AdamW optimizer w/specified parameter groups and epsilon value
optimizer = torch.optim.AdamW(optimizer_grouped_parameters, eps=1e-06)
# Create a OneCycleLR learning rate scheduler
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, pct_start=0.05, anneal_strategy='linear', final_div_factor=10,\
                        max_lr = 5e-4, total_steps = batch_size * num_epochs + 1)
stats = []
result = []
best_val = 0
training_step = 0

In [6]:
# Preparing parallel processing
pool = mp.Pool(num_workers)

# sample nodes:
samp_nodes = np.random.choice(target_nodes, batch_size, replace = False)
sample_depth = 6
sample_width = 520           
inp = {'paper': np.concatenate([samp_nodes, graph.years[samp_nodes]]).reshape(2, -1).transpose()}
seed = randint()
n_batch = 32
batch_size = 128
# node_feature, node_type, edge_time, edge_index, edge_type, (train_mask, valid_mask, test_mask), ylabel = ogbn_sample(seed, samp_nodes, graph, sample_depth, sample_width)

In [7]:
# Sampling Data from MAG
datas = []
print("Starting Sampling...")
for batch_id in np.arange(n_batch):
    print(f'Batch number: {batch_id}')
    node_feature, node_type, edge_time, edge_index, edge_type, (train_mask, valid_mask, test_mask), ylabel = ogbn_sample(seed, samp_nodes, graph, sample_depth, sample_width)
    p = (
        node_feature,
        node_type,
        edge_time,
        edge_index,
        edge_type,
        (train_mask, valid_mask, test_mask),
        ylabel
    )
    datas.append(p)
    print(f'Batch Number: {batch_id}, complete')
print("...Preprocessing complete ")

Starting Sampling...
Batch number: 0


c:\Users\johns\OneDrive\Desktop\HGT_Project\HGT\hgt_utils.py:350: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  node_feature = torch.FloatTensor(node_feature)


Batch Number: 0, complete
Batch number: 1
Batch Number: 1, complete
Batch number: 2
Batch Number: 2, complete
Batch number: 3
Batch Number: 3, complete
Batch number: 4
Batch Number: 4, complete
Batch number: 5
Batch Number: 5, complete
Batch number: 6
Batch Number: 6, complete
Batch number: 7
Batch Number: 7, complete
Batch number: 8
Batch Number: 8, complete
Batch number: 9
Batch Number: 9, complete
Batch number: 10
Batch Number: 10, complete
Batch number: 11
Batch Number: 11, complete
Batch number: 12
Batch Number: 12, complete
Batch number: 13
Batch Number: 13, complete
Batch number: 14
Batch Number: 14, complete
Batch number: 15
Batch Number: 15, complete
Batch number: 16
Batch Number: 16, complete
Batch number: 17
Batch Number: 17, complete
Batch number: 18
Batch Number: 18, complete
Batch number: 19
Batch Number: 19, complete
Batch number: 20
Batch Number: 20, complete
Batch number: 21
Batch Number: 21, complete
Batch number: 22
Batch Number: 22, complete
Batch number: 23
Batch N

In [18]:
# Beginning Model Training
stat = []

# p = (
#     0 node_feature,
#     1 node_type,
#     2 edge_time,
#     3 edge_index,     # An edge_index pair is the same index in the first and second tensors, edge between nodes of that index
#     4 edge_type,      # There is an edge type associated with every edge_index "pair"
#     5 (train_mask, valid_mask, test_mask),
#     6 ylabel
# )

node_feature = datas[0][0]
node_type = datas[0][1]
edge_time = datas[0][2]
edge_index = datas[0][3]
edge_type = datas[0][4]
num_heads = 8


source_type_index = 0
edge_type_index = 4
target_type_index = 0

# edge_mask holds true for all edges of type edge_type_index
edge_mask = (edge_type == int(edge_type_index))

# create mask for all edges that have source node of type source_type_index
source_nodes_mask = (node_type == int(source_type_index))
source_nodes_indexes = source_nodes_mask.nonzero(as_tuple = True)[0] # holds all indexes where a node is of type source_node_type
source_edges_mask = torch.isin(edge_index[0], source_nodes_indexes)

# create mask for all edges that have target nod eof type target_type_index
target_nodes_mask = (node_type == int(target_type_index))
target_nodes_indexes = target_nodes_mask.nonzero(as_tuple = True)[0]
target_edges_mask = torch.isin(edge_index[0], target_nodes_indexes)

# Meta relation triple, True at indexes where typing matches up
meta_relation_mask = edge_mask & source_edges_mask & target_edges_mask

print(meta_relation_mask)
print(len(meta_relation_mask))

source_node_index_location = edge_index[0][meta_relation_mask]
target_node_index_location = edge_index[1][meta_relation_mask]

print(source_node_index_location)
print(len(source_node_index_location))

print(target_node_index_location)
print(len(target_node_index_location))

# get node representaitons based on these indexes



# node_rep = hgt_GNN.forward(node_feature, node_type, edge_index, edge_type, edge_time)

#print(node_rep)

#for node_feature, node_type, edge_time, edge_index, edge_type, (train_mask, valid_mask, test_mask), ylabel in datas:
#    x = 0

tensor([False, False, False,  ..., False, False, False])
104503
tensor([ 568,  471, 1313,  ..., 1716, 1678, 1870])
5242
tensor([   0,    0,    1,  ..., 3244, 3247, 3247])
5242
